In [ ]:
import numpy as np


## 1. SVD とは — 定義と直感

実数 **m × n** 行列 **A** は、常に次の形に分解できます

$$
A \;=\; U\,\Sigma\,V^{\mathsf T},
$$

* **U** … `m × m` の直交行列（列ベクトルは **左特異ベクトル**）
* **Σ** … `m × n` の対角（または “余分を 0 で埋めた対角”）行列。非負の対角要素 **σ₁ ≥ σ₂ ≥ … ≥ σ\_r > 0** が **特異値**
* **V** … `n × n` の直交行列（列ベクトルは **右特異ベクトル**）

行列を──

1. **V** で基底を回転
2. **Σ** で各軸を “伸縮”
3. **U** で再び回転、
   ──したものが **A** だと捉えられます。平面上なら「楕円体を回転して押しつぶす」イメージです。

---

## 2. 主な性質

| 性質  | 概要                                                                                                                                    |
| --- | ------------------------------------------------------------------------------------------------------------------------------------- |
| ランク | 非零特異値の個数 **r = rank(A)**                                                                                                              |
| ノルム | ‖A‖₂ = σ₁（最大特異値）、 ‖A‖\_F² = Σ σᵢ²                                                                                                     |
| 近似  | **Eckart-Young 定理**: 任意の k < r について、  $\displaystyle A_k = \sum_{i=1}^{k} σ_i u_i v_i^{\mathsf T}$  が最小の ‖A − B‖₂ または ‖A − B‖\_F を与える |
| 擬似逆 | Moore–Penrose 逆行列 $A^{+}=V Σ^{+} U^{\mathsf T}$ (Σ の非零要素を逆数に)                                                                         |
| 安定性 | 特異値は小さな摂動に対し連続だが、特異ベクトルは近接した特異値同士で混ざりやすい                                                                                              |

---

## 3. 計算アルゴリズム（概観）

1. **Golub–Kahan bidiagonalization**
2. bidiagonal 行列に **QR 反復**
   → 計算量  $O(mn\min(m,n))$、数値安定性が高い。
   巨大・疎行列では **Lanczos/Arnoldi + ランダム化 SVD** を用いることが多いです。

---

## 4. 固有値分解との関係

* **AᵀA** の固有値: $λ_i = σ_i^{2}$
* **A Aᵀ** も同じ固有値を持つ
* したがって SVD は「左右どちらの Gram 行列も同時に対角化してくれる分解」と見ることができます。

---

## 5. 代表的な応用

| 分野               | 典型的な使い方                                               |
| ---------------- | ----------------------------------------------------- |
| **PCA (主成分分析)**  | 中心化データ行列 X の SVD で主成分を得る (X = UΣVᵀ, 右特異ベクトル=荷重固有ベクトル) |
| **低ランク近似・圧縮**    | 画像を k 枚の “特異画像” に要約、レコメンドでの **matrix factorization**  |
| **ノイズ除去**        | 小さな特異値部分をカットして平滑化                                     |
| **過剰決定/特異な最小二乗** | x = A⁺b が最小ノルム解                                       |
| **数値解析**         | 条件数 κ = σ₁/σ\_r で ill-condition を判定                   |
| **LSA / トピック解析** | 語×文書行列の truncated-SVD (= LSI)                         |
| **制御・信号処理**      | モード解析、Hankel 行列の低ランク化など                               |

---

## 6. Python で試す — 基本操作

```python
import numpy as np

# 例: 6×4 行列
A = np.array([[3, 1, 1, -1],
              [-1, 3, 1, 1],
              [1, 1, 3, 1],
              [1, -1, 1, 3],
              [2, 2, 2, 2],
              [-2, 2, -2, 2]], dtype=float)

U, s, VT = np.linalg.svd(A, full_matrices=False)
Σ = np.diag(s)           # 対角化
# 元に戻るか確認
np.allclose(A, U @ Σ @ VT)   # → True
```

### 低ランク近似 (k = 2)

```python
k = 2
Ak = U[:, :k] @ np.diag(s[:k]) @ VT[:k, :]
error_fro = np.linalg.norm(A - Ak, ord='fro')
```

> **Eckart-Young** により、この Ak が Frobenius ノルム最小の 2 ランク近似です。

---

## 7. 数値的注意点とベストプラクティス

| テーマ        | 実践ポイント                                                             |
| ---------- | ------------------------------------------------------------------ |
| **スケーリング** | 中心化・標準化しないと最大特異値が支配的になり PCA が歪む                                    |
| **縮退特異値**  | ほぼ等しい特異値が並ぶと特異ベクトルが不安定→サブスペースとして扱う                                 |
| **大規模行列**  | `sklearn.utils.extmath.randomized_svd`, `scipy.sparse.linalg.svds` |
| **条件数**    | σ₁/σ\_r が大きいと逆行列計算は不安定。小さな σ\_r を捨てる Tikhonov 正則化が有効               |
| **符号の一意性** | (uᵢ, vᵢ) の同時符号反転は同じ分解。アプリによって “符号整列” が必要                           |

---

## 8. まとめ — SVD を使いこなすコツ

1. **構造の分離**: 幾何学的 “回転-伸縮-回転” として直感を掴む。
2. **ランク r vs k**: 応用目的（圧縮・可視化・ノイズ除去）に合わせて k を選択。
3. **条件数で健全性チェック**: ill-condition なら正則化 or 低ランク近似。
4. **高速版を活用**: 行列サイズと疎密に応じて deterministic / randomized / Lanczos を切替え。
5. **特異ベクトルを読み解く**: PCA なら荷重方向、レコメンドなら潜在因子に解釈を与えると洞察が深まる。




In [ ]:
#81 SVDの基本
A=np.array([
    [1,2,3],
    [3,4,5]
])

U,s,Vt=np.linalg.svd(A,full_matrices=False)
sigma=np.diag(s)
print(U)
print(sigma)
print(Vt)
print(U@sigma@Vt)
np.allclose(A,U@sigma@Vt)

B=np.array([
    [1,2,3,4],
    [3,4,5,1],
    [2,3,4,5],
    [1,3,5,7]
])

U,s,Vt=np.linalg.svd(B,full_matrices=False)
sigma=np.diag(s)
print(U)
print(sigma)
print(Vt)
print(U@sigma@Vt)
np.allclose(B,U@sigma@Vt)

In [ ]:
#82特異点と固有値の関係
At=A.T
eigval,eigvec=np.linalg.eig(At@A)#固有値と固有ベクトル
print(eigval)
U,s,Vt=np.linalg.svd(A,full_matrices=False)#sが特異値
sigma=np.diag(s)
print(sigma*sigma)#特異値を2乗

Bt=B.T
eigval,eigvec=np.linalg.eig(Bt@B)
print(eigval)
U,s,Vt=np.linalg.svd(B,full_matrices=False)
sigma=np.diag(s)
print(sigma*sigma)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#83低ランク近似
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# ---- 画像読み込み --------------------------------------------------
img = plt.imread('output.png')          # (H,W,3) or (H,W,4)

# αチャネル除去
if img.ndim == 3 and img.shape[2] == 4:
    img = img[..., :3]

# グレースケール化
gray = img.mean(axis=2)                 # 0–1 or 0–255

# 0–255 スケーリング
if gray.max() <= 1.0:
    gray = gray * 255

A = gray.astype(float)

# ---- SVD -----------------------------------------------------------
U, s, Vt = np.linalg.svd(A, full_matrices=False)

def low_rank(U, s, Vt, k):
    Sk = np.diag(s[:k])
    return U[:, :k] @ Sk @ Vt[:k, :]

k = 20      # ← まず 100 くらいで試す
B = low_rank(U, s, Vt, k)

# ---- 可視化 --------------------------------------------------------
fig, ax = plt.subplots(1, 2, figsize=(8,4))
ax[0].set_title('Original'); ax[0].imshow(gray, cmap='gray'); ax[0].axis('off')
ax[1].set_title(f'Rank {k}'); ax[1].imshow(np.clip(B,0,255).astype(np.uint8),
                                           cmap='gray'); ax[1].axis('off')
plt.show()

# ---- 保存 (任意) ---------------------------------------------------
Image.fromarray(np.clip(B,0,255).astype(np.uint8)).save(f'rank{k}.png')


In [ ]:
#84低ランク近似
k=1
for i in range(10):
    k*=2
    B = low_rank(U, s, Vt, k)

    # ---- 可視化 --------------------------------------------------------
    fig, ax = plt.subplots(1, 2, figsize=(8,4))
    ax[0].set_title('Original'); ax[0].imshow(gray, cmap='gray'); ax[0].axis('off')
    ax[1].set_title(f'Rank {k}'); ax[1].imshow(np.clip(B,0,255).astype(np.uint8),
                                               cmap='gray'); ax[1].axis('off')
    plt.show()

In [69]:
#85行列の条件数
# ---- 画像読み込み --------------------------------------------------
img = plt.imread('output.png')          # (H,W,3) or (H,W,4)
print(img.shape)

gray = img.mean(axis=2)                 # 0–1 or 0–255
print(gray.shape)
# 0–255 スケーリング
if gray.max() <= 1.0:
    gray = gray * 255

A = gray.astype(float)
# ---- SVD -----------------------------------------------------------
U, s, Vt = np.linalg.svd(A, full_matrices=False)
print('最大特異値',max(s))
print('最小特異値',min(s))

(706, 656, 4)
(706, 656)
最大特異値 124612.74612797872
最小特異値 6.9267612465156965e-12


Moore–Penrose の擬似逆行列 (pseudo-inverse) は、理論的には以下の 4 つの条件（Moore–Penrose 条件）を同時に満たす一意の行列として定義されます。実装で一般的に採用されているアルゴリズムは、**SVD (特異値分解)** を利用した方法です。

---

# 1. Moore–Penrose 逆行列の定義

ある行列 $A$ に対し、行列 $A^+$ が Moore–Penrose 逆行列であるとは、

1. $A A^+ A = A$
2. $A^+ A A^+ = A^+$
3. $(A A^+)^T = A A^+$
4. $(A^+ A)^T = A^+ A$

の 4 つの条件を満たす一意の行列 $A^+$ のことです。

行列 $A$ が正則（平方・フルランク）なら単に $A^+ = A^{-1}$ ですが、そうでない場合でも擬似逆行列は定義できます。**ランク落ち** していても、最小二乗解を与えるなどの有用な性質があります。

---

# 2. SVD (特異値分解) を用いた擬似逆行列の構成

## 2.1 特異値分解 (SVD)

行列 $A \in \mathbb{R}^{m \times n}$ に対して SVD を行うと、通常は下記のように分解できます。

$$
A = U \, \Sigma \, V^T
$$

* $U$: $m \times m$ の直交行列 (列ベクトルが左特異ベクトル)
* $\Sigma$: $m \times n$ の対角成分が特異値（非負）の行列で、対角以外は 0
* $V$: $n \times n$ の直交行列 (列ベクトルが右特異ベクトル)

数値計算的には「薄い SVD（thin SVD）」や「エコノミーサイズ SVD」と呼ばれる形 (例えば $m \times r,\; r \times r,\; r \times n$ の分解) を使うことが多いですが、概念的には上記のサイズでよいです。

$\Sigma$ の形は例えば：

$$
\Sigma = 
\begin{pmatrix}
\sigma_{1} & 0         & \cdots & 0 \\
0         & \sigma_{2} & \cdots & 0 \\
\vdots    & \vdots    & \ddots & \vdots \\
0         & 0         & \cdots & \sigma_{p} \\
0         & 0         & \cdots & 0 \\
\vdots    & \vdots    & \ddots & \vdots 
\end{pmatrix}
$$

ただし $p = \min(m, n)$ で、$\sigma_{1} \ge \sigma_{2} \ge \dots \ge \sigma_{p} \ge 0$ が特異値です。

## 2.2 擬似逆行列の式

SVD で $A = U \Sigma V^T$ と表せたら、**Moore–Penrose 擬似逆行列** $A^+$ は下記のように与えられます。

$$
A^+ = V \, \Sigma^+ \, U^T
$$

ここで $\Sigma^+$ は $\Sigma$ の **対角成分（特異値）のうち 0 でないものをその逆数に置き換えた対角行列** です。
例えば、$\Sigma$ の対角成分が $\sigma_i$ (もし 0 の特異値があれば無視・または逆数 0 とする) のとき、$\Sigma^+$ の対角成分は

$$
\begin{cases}
\frac{1}{\sigma_i}, & \sigma_i > 0 \\
0, & \sigma_i = 0
\end{cases}
$$

となります。

### 数値的にはしきい値を設ける

実装では $\sigma_i$ が非常に小さいとき、計算上の精度を考慮して「0 とみなす」こともあります。
Python の `np.linalg.pinv` は内部的に SVD を行い、**ある小さなしきい値以下の特異値を 0 とみなす**ことで安定性を確保しています。

---

# 3. 実装例 (擬似コード)

以下は、NumPy の `np.linalg.pinv` とほぼ同等の処理を **疑似コード** で表したものです。
（実際には `np.linalg.svd` は LAPACK 等を呼び出して最適化されているため、もっと複雑です。）

```python
import numpy as np

def pseudo_inverse_via_svd(A, rcond=1e-15):
    """
    A の Moore-Penrose 擬似逆行列を SVD を用いて近似的に計算する。
    rcond: 特異値のカットオフ（相対的なしきい値）
    """
    # 1. 特異値分解
    U, s, Vt = np.linalg.svd(A, full_matrices=False)
    # A.shape = (m, n)
    # U.shape = (m, k),  s.shape = (k,),  Vt.shape = (k, n)
    # ここで k = min(m, n)

    # 2. 特異値のしきい値決定
    #    s[0] が最大特異値と想定 (昇順/降順は実装によって異なることも)
    cutoff = rcond * np.max(s)

    # 3. s の逆数をとる（ただし小さいものは 0 にする）
    s_inv = np.array([1.0/val if val > cutoff else 0.0 for val in s])

    # 4. V, Sigma^+, U^T の積をとる
    # Vt = V^T なので V = (Vt)^T
    V = Vt.T
    # Sigma^+ は s_inv を対角成分とする行列
    # ただし U, V との乗算を考慮して 2次元に変形
    S_plus = np.diag(s_inv)

    # 5. A^+ = V * S^+ * U^T
    A_pinv = V @ S_plus @ U.T

    return A_pinv
```

* `np.linalg.svd(A, full_matrices=False)` で **薄い SVD** を計算しています。

  * `A.shape = (m,n)` に対して

    * `U.shape = (m,k)`
    * `s.shape = (k,)`
    * `Vt.shape = (k,n)`
    * ここで `k = min(m, n)`。
* `rcond` は、特異値が最大特異値の `rcond` 倍未満であればゼロとみなすための相対的なしきい値。

  * `np.linalg.pinv` のドキュメントによれば、既定値は `1e-15` に近い値だったり、バージョンによって微調整がされています。
* 計算結果の `A_pinv` が **Moore–Penrose 擬似逆行列** となり、ランク落ちしている場合でも最小二乗的に最善な逆を与えてくれます。

---

# 4. まとめ

1. **Moore–Penrose 逆行列** は、ランク落ちを含めどんな行列でも定義できる「一般化された逆行列」であり、最小二乗解や最小ノルム解などで多用される。
2. **実装には SVD を用いる** ことが多く、

   * $A = U \Sigma V^T$
   * $A^+ = V \Sigma^+ U^T$  （$\Sigma^+$ は非零特異値の逆数を並べた対角行列）
     という手順で求める。
3. 数値計算では、しきい値以下の特異値を 0 として逆をとらないようにする（ノイズへの安定性や病的条件数を緩和するため）。

これが、**「擬似逆行列を作成するアルゴリズム」＝ SVD に基づいた Moore–Penrose 逆行列構成法** の概要です。実際のライブラリ `np.linalg.pinv` も基本的にはこのアプローチを採っています。


In [80]:

#86 擬似逆行列
#import numpy as np

A = np.array([
    [1,  2,  3],
    [2,  4,  6],  # 1行目の2倍
    [1, -1,  0],
    [1, -1,  0]
], dtype=float)

# A のランクを確認
print("A.shape:", A.shape)
print("rank(A):", np.linalg.matrix_rank(A))

# --- 方程式 A x = b を考える ---------------------------------------
b = np.array([1, 2, 3, 4], dtype=float)

# --- 擬似逆行列を使った最小二乗解 -----------------------------------
A_pinv = np.linalg.pinv(A)      # Moore-Penrose の擬似逆行列
x = A_pinv @ b                  # x = (A^+) b

print("解 x =")
print(x)

# --- Ax と b を比較して誤差をチェック -------------------------------
Ax = A @ x
residual = np.linalg.norm(Ax - b)
print("A x =", Ax)
print("b    =", b)
print(f"ノルム誤差: {residual}")


A.shape: (4, 3)
rank(A): 2
解 x =
[ 2.05555556 -1.44444444  0.61111111]
A x = [1.  2.  3.5 3.5]
b    = [1. 2. 3. 4.]
ノルム誤差: 0.7071067811865476


In [92]:
#87 最小二乗法
A2=np.array([
    [1,2,3],
    [4,5,6],
    [7,8,9]
])
b = np.array([2, 3, 4], dtype=float)
A2t=A2.T
print(A2t)
C=np.linalg.inv(A2t@A2)
print(C)
x=C@A2t@b
print(x)

[[1 4 7]
 [2 5 8]
 [3 6 9]]
[[-3.51843721e+13  7.03687442e+13 -3.51843721e+13]
 [ 7.03687442e+13 -1.40737488e+14  7.03687442e+13]
 [-3.51843721e+13  7.03687442e+13 -3.51843721e+13]]
[-0.96875  0.       1.     ]


In [102]:
#88　最小二乗法
A2=np.array([
    [1,2,3],
    [2,4,6],
    [7,8,9]
])
print(A2.shape)
print(np.linalg.matrix_rank(A2))
b = np.array([2, 3, 4], dtype=float)
C=np.linalg.pinv(A2)
print(C)
x=C@b
print(x)

(3, 3)
2
[[-0.14444444 -0.28888889  0.22222222]
 [-0.01111111 -0.02222222  0.05555556]
 [ 0.12222222  0.24444444 -0.11111111]]
[-0.26666667  0.13333333  0.53333333]


In [ ]:
#89SVD によるランクの解釈
from collections import Counter
r=[]
for _ in range(100000):
    A2 = np.random.randint(0, 5, size=(4, 4)) 
    U,s,Vt=np.linalg.svd(A2)
    count=sum(e<0.2 for e in s)
    rank=np.linalg.matrix_rank(A2)
    r.append((int(count),int(rank)))
print(Counter(r))

Counter({(0, 4): 78263, (1, 4): 15616, (1, 3): 6059, (2, 3): 35, (2, 2): 27})
